<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Take-notice!" data-toc-modified-id="Take-notice!-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Take notice!</a></span></li><li><span><a href="#Multiple-overlays" data-toc-modified-id="Multiple-overlays-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Multiple overlays</a></span><ul class="toc-item"><li><span><a href="#LA-Times-Neighborhoods" data-toc-modified-id="LA-Times-Neighborhoods-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LA Times Neighborhoods</a></span></li><li><span><a href="#Get-Crime-Data-from-LA-Open-Data-Portal" data-toc-modified-id="Get-Crime-Data-from-LA-Open-Data-Portal-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get Crime Data from LA Open Data Portal</a></span></li><li><span><a href="#Convert-data-to-a-geodataframe" data-toc-modified-id="Convert-data-to-a-geodataframe-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Convert data to a geodataframe</a></span></li><li><span><a href="#Create-a-two-layer-map" data-toc-modified-id="Create-a-two-layer-map-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Create a two layer map</a></span></li><li><span><a href="#Zoom-to-the-points,-not-the-neighborhoods" data-toc-modified-id="Zoom-to-the-points,-not-the-neighborhoods-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Zoom to the points, not the neighborhoods</a></span></li><li><span><a href="#Joining-crime-to-neighborhoods" data-toc-modified-id="Joining-crime-to-neighborhoods-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Joining crime to neighborhoods</a></span></li><li><span><a href="#The-spatial-join" data-toc-modified-id="The-spatial-join-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>The spatial join</a></span></li><li><span><a href="#Join-the-value-counts-back-to-the-neighborhoods" data-toc-modified-id="Join-the-value-counts-back-to-the-neighborhoods-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Join the value counts back to the neighborhoods</a></span></li><li><span><a href="#Map-neighborhoods-by-crime-count" data-toc-modified-id="Map-neighborhoods-by-crime-count-2.9"><span class="toc-item-num">2.9&nbsp;&nbsp;</span>Map neighborhoods by crime count</a></span></li><li><span><a href="#Cleanup:-Adding-basemaps,-titles..." data-toc-modified-id="Cleanup:-Adding-basemaps,-titles...-2.10"><span class="toc-item-num">2.10&nbsp;&nbsp;</span>Cleanup: Adding basemaps, titles...</a></span></li><li><span><a href="#Make-an-interactive-map" data-toc-modified-id="Make-an-interactive-map-2.11"><span class="toc-item-num">2.11&nbsp;&nbsp;</span>Make an interactive map</a></span></li><li><span><a href="#Get-the-center-lat/lon" data-toc-modified-id="Get-the-center-lat/lon-2.12"><span class="toc-item-num">2.12&nbsp;&nbsp;</span>Get the center lat/lon</a></span></li><li><span><a href="#Saving-as-HTML" data-toc-modified-id="Saving-as-HTML-2.13"><span class="toc-item-num">2.13&nbsp;&nbsp;</span>Saving as HTML</a></span></li></ul></li></ul></div>

<div class="alert alert-danger">

<h1>Take notice!</h1>
<ul>
    <li>Make sure you are working with a copy and not the original notebook file</li>
    <li>This class will be recorded</li>
</ul>
    
</div>

# Multiple overlays

<img src="images/mult.png">

We have focused on data exploration on single dataframes, geared to create map visualizations for individual layers. This lab goes over *overlays*, creating a single map that has multiple layers of data.

Research inquiry: Which neighborhoods in Los Angeles have the highest instances of arrests?

## LA Times Neighborhoods

* [About mapping LA neighborhoods](http://maps.latimes.com/neighborhoods/)
* [Download the data](http://boundaries.latimes.com/sets/)

In [ ]:
import geopandas as gpd

In [ ]:
# get neighborhood boundaries from the LA Times
neighborhoods = gpd.read_file('http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-v5.geojson')

In [ ]:
# trim the data to the bare minimum columns
neighborhoods = neighborhoods[['name','geometry']]
neighborhoods.head()

In [ ]:
# plot it!
ax=neighborhoods.plot(figsize=(12,12),color='gainsboro', edgecolor='white')

## Get Crime Data from LA Open Data Portal
Next, we acquire the data using the socrata API. Use the socrata documentation to grab the code syntax for our crime data.
- https://dev.socrata.com/foundry/data.lacity.org/amvf-fr72

In [ ]:
import pandas as pd
import plotly.express as px
from sodapy import Socrata

In [ ]:
# connect to the data portal
client = Socrata("data.lacity.org", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("amvf-fr72", 
                     limit=5000,
                     where = "arst_date between '2020-09-01T00:00:00' and '2020-09-30T00:00:00'",
                     order='arst_date desc')

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

# print it with .sample, which gives you random rows
df.head()

In [ ]:
df.shape

In [ ]:
# columns
list(df)

## Convert data to a geodataframe

Geopandas allows us to convert different types of data into a spatial format.
- https://geopandas.org/gallery/create_geopandas_from_pandas.html

In [ ]:
# convert pandas dataframe to geodataframe
crime = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

In [ ]:
# map it!
crime.plot(figsize=(12,12),color='red')

## Create a two layer map

- https://geopandas.org/mapping.html

In [ ]:
# first define which layers will be your "base"
base = neighborhoods.plot(figsize=(12,10),color='gainsboro', edgecolor='white')

# define the layer that will go on top, and add the base layer to the `ax` argument
ax = crime.plot(ax=base, color='red', markersize=5)

## Zoom to the points, not the neighborhoods

By default, the map has zoomed out to fit both layers. Sometimes, we may want to zoom to the extent of a smaller layer, in this case, the crime data.

In [ ]:
# get the bounding box coordinates for the crime data
crime.geometry.total_bounds

In [ ]:
# shortcut to put them into their own variables
minx, miny, maxx, maxy = crime.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)


In [ ]:
# use the bounding box coordinates to set the x and y limits
base = neighborhoods.plot(figsize=(12,12),color='gainsboro', edgecolor='white')
ax = crime.plot(ax=base, marker='o', color='red', markersize=5)
ax.set_xlim(minx - .1, maxx + .1) # added/substracted value is to give some margin around total bounds
ax.set_ylim(miny - .1, maxy + .1)
ax

## Joining crime to neighborhoods

Now that we have successfuly overlaid our two layers on a single map, we want to get some correlations between them. We do so first by conducting a spatial join that will allow us to summarize the number of crime instances per neighborhood.

First things first, whenever we do spatial analysis between two layers, make sure they are in the same projection.

In [ ]:
# set the coordinate system to WGS84 for our crime data
crime.set_crs(epsg=4326, inplace=True)
crime.crs

In [ ]:
# check the coordinate system of our neighborhoods
neighborhoods.crs

## The spatial join

* https://geopandas.org/mergingdata.html?highlight=spatial%20join

In a Spatial Join, two geometry objects are merged based on their spatial relationship to one another.

The how argument specifies the type of join that will occur and which geometry is retained in the resultant geodataframe. It accepts the following options:

`left`: use the index from the first (or left_df) geodataframe that you provide to sjoin; retain only the left_df geometry column

`right`: use index from second (or right_df); retain only the right_df geometry column

`inner`: use intersection of index values from both geodataframes; retain only the left_df geometry column



In [ ]:
# Do the spatial join
join = gpd.sjoin(neighborhoods,
                 crime,
                 how='right')

In [ ]:
# Now every instance of crime is given a neighborhood it falls inside of
join.head()

Next, we create a dataframe that counts crime by neighborhood:

In [ ]:
crime_by_neighborhoods = join.name.value_counts().rename_axis('name').reset_index(name='crime_count')

In [ ]:
crime_by_neighborhoods.head()

In [ ]:
# make a bar chart
crime_by_neighborhoods[:50].plot.bar(figsize=(20,8),x='name',y='crime_count')

## Join the value counts back to the neighborhoods

The bar chart is nice, but what we also want is a choropleth map to accompany it. To do so, we merge the counts back to the neighborhoods.

In [ ]:
# join the summary table back to the neighborhood geodatabase
neighborhoods=neighborhoods.merge(crime_by_neighborhoods,on='name')

In [ ]:
# our neighborhood table now has a count column
neighborhoods.head()

## Map neighborhoods by crime count

In [ ]:
# plot it!
neighborhoods.plot(figsize=(15,15),column='crime_count',legend=True,cmap='RdYlGn_r')

## Cleanup: Adding basemaps, titles...

In [ ]:
# for basemaps
import contextily as ctx

In [ ]:
# reproject to web mercator
neighborhoods = neighborhoods.to_crs(epsg=3857)

In [ ]:
ax = neighborhoods.plot(figsize=(15,15),
                        column='crime_count',
                        legend=True,
                        alpha=0.8,
                        cmap='RdYlGn_r')

ax.axis('off')
ax.set_title('September 2020 Arrests by the LAPD',fontsize=22)
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

## Make an interactive map

To create an interactive version of the same map, let's use plotly express. Since we projected our data to web mercator, note that we have to project it back to WGS84 to work with plotly.

In [ ]:
import plotly.express as px

In [ ]:
# reproject to web mercator
neighborhoods = neighborhoods.to_crs(epsg=4326)

In [ ]:
neighborhoods.crs

## Get the center lat/lon

Plotly maps requires you to give it center coordinates. Let's calculate this based on the data.

First, get the bounding box coordinates of the neighborhoods:

In [ ]:
minx, miny, maxx, maxy = neighborhoods.geometry.total_bounds

In [ ]:
center_lat=(maxy-miny)/2+miny
center_lat

In [ ]:
center_lon=(maxx-minx)/2+minx
center_lon

In [ ]:
# map it!
fig = px.choropleth_mapbox(neighborhoods, 
                           geojson=neighborhoods.geometry, 
                           locations=neighborhoods.index, 
                           color='crime_count',
                           color_continuous_scale="rdylgn_r",
                           mapbox_style="carto-positron",
                           zoom=9, 
                           center = {"lat": center_lat, "lon": center_lon},
                           hover_name=neighborhoods.name,
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Saving as HTML

In [ ]:
# save it
fig.write_html("lacrime.html")